In [1]:
import torch

In [2]:
from comet_ml import Experiment, Optimizer

In [3]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from collections import defaultdict
from tqdm import trange
import torch
import seaborn as sns
import os
import matplotlib.pyplot as plt
from io import BytesIO
import base64
import sklearn
import time
import datetime

plt.style.use("seaborn")
torch.set_default_dtype(torch.float32)

In [4]:
import datetime


In [5]:
import node

In [6]:
from sklearn.metrics import roc_auc_score, f1_score, precision_score, recall_score, balanced_accuracy_score

In [7]:
from tabular_hypernet import HypernetworkPCA, TrainingModes, Hypernetwork
from tabular_hypernet.modules import SimpleNetwork
from tabular_hypernet.training_utils import train_slow_step, train_model, train_carthesian
from tabular_hypernet.interfaces import HypernetworkSklearnInterface, SimpleSklearnInterface
# from ipynb.fs.defs.MNIST_benchmark import test_model
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

In [8]:
from sklearn.metrics import classification_report, balanced_accuracy_score

In [9]:
import imblearn

In [10]:
import pyhopper

In [11]:
from loguru import logger

logger.add("log_Lymf.txt", format='{time:YYYY-MM-DD HH:mm:ss.SSS} | {message}')

1

In [12]:
import seaborn as sns

In [13]:
DEVICE="cuda:0"

In [14]:
from loguru import logger

logger.add("log.txt", format='{time:YYYY-MM-DD HH:mm:ss.SSS} | {message}')

2

In [15]:
GS_METRIC = "balanced_accuracy"

# Load data

In [16]:
DATA = "Hill-Valley-with-noise"

In [17]:
if DATA == "Hill-Valley-with-noise":
    hill_valley_train = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/hill-valley/Hill_Valley_with_noise_Training.data')
    hill_valley_test = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/hill-valley/Hill_Valley_with_noise_Testing.data')
    
    X_train = hill_valley_train.values[:, :-1].astype(float)
    y_train = hill_valley_train.values[:, -1]
    y_train = LabelEncoder().fit_transform(y_train).astype(int)
    
    X_test = hill_valley_test.values[:, :-1].astype(float)
    y_test = hill_valley_test.values[:, -1]
    y_test = LabelEncoder().fit_transform(y_test).astype(int)
    
    print('train', X_train.shape, len(np.unique(y_train)))
    print('test', X_test.shape, len(np.unique(y_test)))
    
    X = (X_train, X_test)
    y = (y_train, y_test)


train (606, 100) 2
test (606, 100) 2


In [18]:
n_classes = len(np.unique(y if not isinstance(y, tuple) else y_train))
n_classes

2

In [19]:
unique, counts = np.unique(y if not isinstance(y, tuple) else y_train, return_counts=True)

print(dict(zip(unique, counts)))

{0: 307, 1: 299}


In [20]:
n_features = X.shape[1] if not isinstance(X, tuple) else X_train.shape[1]
n_features

100

In [21]:
max_size = int(len(X)*0.7) if not isinstance(X, tuple) else len(X_train)

In [22]:
max_size

606

In [23]:
# print('dataset size:', len(X), '|', 'max training size:', max_size)

In [24]:
TEST_RUN = False

# Preprocess

In [25]:
np.random.seed(42)

def prepare_data(X, y, size=None):
    if isinstance(X, tuple) and isinstance(y, tuple):
        X_train, X_test = X
        y_train, y_test = y
    else:    
        X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=size, stratify=y)
    # X_train, y_train = imblearn.over_sampling.RandomOverSampler(random_state=42).fit_resample(X_train, y_train)
    
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)
    
    X_train, X_test, y_train, y_test = [torch.from_numpy(x) for x in [X_train, X_test, y_train, y_test]]
    
    return X_train, X_test, y_train, y_test

In [26]:
X_train, X_test, y_train, y_test = prepare_data(X, y, size=max_size)

## Benchmark

In [27]:
criterion = torch.nn.CrossEntropyLoss()

In [28]:
def _summarize_results(y_pred, y_score, y_test, labels):
    results = []
    for idx, label in enumerate(labels):
        y_pred_filt = y_pred[y_test==idx]
        y_test_filt = y_test[y_test==idx]
        acc = (y_pred_filt==y_test_filt.numpy()).sum()/len(y_test_filt)*100
        results.append({
            "Class": label,
            "Metric": acc
        })
        
    acc = (y_pred==y_test.numpy()).sum()/len(y_test)*100    
    results.append({
        "Class": "Total",
        "Metric": acc
    })
    
    
    results.append({
        "Class": "balanced_accuracy",
        "Metric": balanced_accuracy_score(y_test, torch.from_numpy(y_pred)).item()*100
    })
    
    try:
        results.append({
            "Class": "F1 score",
            "Metric": f1_score(y_test, torch.from_numpy(y_pred)).item()*100
        })
        results.append({
            "Class": "roc_auc",
            "Metric": roc_auc_score(y_test, torch.from_numpy(y_score[:, 1])).item()*100
        })
        results.append({
            "Class": "Precision",
            "Metric": precision_score(y_test, torch.from_numpy(y_pred)).item()*100
        })
        results.append({
            "Class": "Recall",
            "Metric": recall_score(y_test, torch.from_numpy(y_pred)).item()*100
        })
    except ValueError:
        pass
    return results

def test_model(model_fn, data, train_size, label_encoder=None, iters=10, as_numpy=False):
    if TEST_RUN:
        iters = 1
        
    if label_encoder is not None:
        labels = label_encoder.classes_
    else:
        labels = sorted(pd.unique(data[1][0] if isinstance(data[1], tuple) else data[1]))
    
    results = []

    for i in range(iters):
        print('iter', i+1, 'of', iters)
        X_train, X_test, y_train, y_test = prepare_data(*data, train_size)
        
        model = model_fn()

        if as_numpy:
            model.fit(X_train.numpy(), y_train.numpy());
        else:
            model.fit(X_train, y_train)
        
        y_pred = model.predict(X_test)
        y_score = model.predict_proba(X_test)
        results.extend(_summarize_results(y_pred, y_score, y_test, labels))
        
#         del model
#         torch.cuda.empty_cache()

    dframe = pd.DataFrame.from_dict(results)
#     sns.violinplot(data=dframe[dframe["Class"]!="Loss"], y="Class", x="Metric", orient='h')
    return dframe

### Param search

In [29]:
# def grid_search_best_params(model_fn, param_grid, data_size=max_size, scoring=GS_METRIC):
#     if TEST_RUN:
#         tmp = {k: [v[0]] for k, v in param_grid.items()}
#         param_grid = tmp
        
#     X_train, X_test, y_train, y_test = prepare_data(X, y, data_size)

#     cv_clf = GridSearchCV(
#         estimator=model_fn(), 
#         param_grid=param_grid,
#         scoring=scoring, 
#         return_train_score=True,
#         verbose=1, 
#         cv=5,
#     )

#     res = cv_clf.fit(X_train, y_train)

#     print(f'best params for {DATA}', res.best_params_)
#     print('mean_train_score', cv_clf.cv_results_['mean_train_score'].mean())
#     print('std_train_score', cv_clf.cv_results_['std_train_score'].mean())

#     predictions = cv_clf.predict(X_test) 
#     print(classification_report(y_test, predictions))
    
#     with open(f"{DATA}_{model_fn.__name__}_best_params.txt", "a") as f:
#             f.write(str(res.best_params_) + ", " + str(balanced_accuracy_score(y_test, predictions)) + "\n")
    
#     print(f"{DATA}_{model_fn.__name__}_{res.best_params_}")
    
#     return res.best_params_

In [30]:
def pyhopper_best_params(model_fn, param_grid, data_size=max_size, metric=GS_METRIC, time="30m", default_params={}):
    if TEST_RUN:
        time = 30
        if 'epochs' in param_grid:
            param_grid["epochs"] = pyhopper.choice([10])
        
    logger.info(f"pyhopper_best_params {model_fn.__name__}")
    
    def objective(params):
#         torch.cuda.empty_cache()

        model_results = test_model(
                        model_fn(
                            **default_params,
                            **params
                        ),
                        (X, y),
                        data_size,
                        None, 5)
        with open(f"{DATA}_{model_fn.__name__}_params.txt", "a") as f:
            f.write(str(datetime.datetime.now()) + ", " + str(params) + ", " + str(model_results[model_results["Class"]==metric]["Metric"].mean()) + "\n")
        return model_results[model_results["Class"]==metric]["Metric"].mean()

    from pyhopper.callbacks import History
    search = pyhopper.Search(param_grid)

    best_params = search.run(objective, "maximize", time, n_jobs="1x per-gpu", seeding_ratio=0.5)
    
    with open(f"{DATA}_{model_fn.__name__}_best_params.txt", "a") as f:
            f.write(str(best_params))
    
    print(f"{DATA}_{model_fn.__name__}_{best_params}")
    return best_params, search.history


In [31]:

datetime.datetime.now()


datetime.datetime(2023, 1, 11, 19, 28, 1, 373575)

# TRAIN MODELS

### XGBoost

In [32]:
from xgboost import XGBClassifier

In [33]:
import xgboost
xgboost.__version__

'1.6.0'

In [34]:
def get_xgboost(**params):
    random_seed = np.random.randint(1024)
    def _inner(**args):
        return XGBClassifier(
            verbosity=0,
            random_state=random_seed,
            use_label_encoder=False,
            **params,
            **args
        )
    return _inner    

#### Hyperparam tuning

In [35]:
param_grid = {
                'n_estimators': pyhopper.int(50, 3000, multiple_of=50),
                'max_depth': pyhopper.choice([2, 3, 5, 10, 15]),
                'learning_rate': pyhopper.float(1e-5,1e-1, log=True),
                'min_child_weight': pyhopper.choice([1, 2, 4, 8, 16, 32]),
                'gamma': pyhopper.choice([0, 0.001, 0.1, 1]),
             }

xgbt_best1, xgbt_history1 = pyhopper_best_params(get_xgboost, param_grid, time="90m")

2023-01-11 19:28:01.519 | INFO     | __main__:pyhopper_best_params:7 - pyhopper_best_params get_xgboost


  0%|          | [00:00<?]

Search is scheduled for 01:30:00 (h:m:s)
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5

iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter

In [36]:
param_grid = {
                'subsample': pyhopper.choice([0.5, 0.6, 0.7, 0.8, 0.9, 1]),
                'reg_lambda': pyhopper.float(1e-5, 10, init=0, log=True),
                'reg_alpha': pyhopper.float(1e-5, 10, init=0, log=True),
             }


xgbt_best2, xgbt_history2 = pyhopper_best_params(get_xgboost, param_grid, time="90m", default_params=xgbt_best1)

2023-01-11 20:58:07.479 | INFO     | __main__:pyhopper_best_params:7 - pyhopper_best_params get_xgboost


  0%|          | [00:00<?]

Search is scheduled for 01:30:00 (h:m:s)
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5

#### Best Params

'Ionosphere' {'gamma': 0, 'learning_rate': 0.1, 'max_depth': 5, 'min_child_weight': 2, 'n_estimators': 1500}


In [37]:
xgboost_best = {**xgbt_best1, **xgbt_best2}

In [38]:
data_size = max_size

xgb_dframe = test_model(get_xgboost(**xgboost_best),
                        (X, y),
                        data_size,
                        label_encoder=None,
                        iters=10)

iter 1 of 10
iter 2 of 10
iter 3 of 10
iter 4 of 10
iter 5 of 10
iter 6 of 10
iter 7 of 10
iter 8 of 10
iter 9 of 10
iter 10 of 10


In [39]:
res = xgb_dframe[xgb_dframe["Class"]=="Total"].reset_index(drop=True)["Metric"]
print(f"{data_size}: {res.mean():.2f} ~ {res.std():.2f} (max: {res.max():.2f})")

606: 57.76 ~ 0.00 (max: 57.76)


In [40]:
xgb_dframe.groupby(['Class']).mean()

,Metric
Class,
0,60.869565
1,54.723127
F1 score,56.756757
Precision,58.947368
Recall,54.723127
Total,57.755776
balanced_accuracy,57.796346
roc_auc,61.908315


### NODE

In [41]:
from qhoptim.pyt import QHAdam

In [42]:
def node_fn(layer_dim=128, num_layers=1, depth=3):
    def _inner():
        network = torch.nn.Sequential(
            node.DenseBlock(X_train.shape[1], 
                            layer_dim=layer_dim,
                            num_layers=num_layers, 
                            tree_dim=n_classes+1, 
                            depth=depth, 
                            flatten_output=False,
                            choice_function=node.entmax15, 
                            bin_function=node.entmoid15
                           ),
            node.Lambda(lambda x: x.mean(dim=1))
        )
        
        
        network = network.to(DEVICE)
        network.device=DEVICE
        
        with torch.no_grad():
            res = network(torch.as_tensor(X_train, device=DEVICE).to(torch.float32))
        
            
        optimizer_params = { 'nus':(0.7, 1.0), 'betas':(0.95, 0.998) }
        optim = QHAdam(network.parameters(), **optimizer_params)
            
        network = SimpleSklearnInterface(network, device=DEVICE, epochs=150, batch_size=32)
        network.optimizer = optim
        return network
    return _inner

#### Tune hyperparams

In [43]:
# 'layer_dim': hp.quniform('layer_dim', 100, 1200, 100),
# 'num_layers': hp.quniform('num_layers', 1, 4, 1),
# 'depth': hp.quniform('depth', 2, 7, 1)
                    
param_grid = {
    'layer_dim': pyhopper.int(64, 1024, power_of=2),
    'num_layers': pyhopper.int(1, 5),
    'depth': pyhopper.int(2, 7),
}

node_best, node_history = pyhopper_best_params(node_fn, param_grid, time="180m")
node_best

2023-01-11 22:29:51.891 | INFO     | __main__:pyhopper_best_params:7 - pyhopper_best_params node_fn


  0%|          | [00:00<?]

Search is scheduled for 03:00:00 (h:m:s)
iter 1 of 5


/home/z1157095/hypernet-cnn/deps/node/node/lib/odst.py:72: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  bin_codes = (indices.view(1, -1) // offsets.view(-1, 1) % 2).to(torch.float32)
/home/z1157095/hypernet-cnn/deps/node/node/lib/odst.py:113: UserWarning: Data-aware initialization is performed on less than 1000 data points. This may cause instability.To avoid potential problems, run this model on a data batch with at least 1000 data samples.You can do so manually before training. Use with torch.no_grad() for memory efficiency.
  warn("Data-aware initialization is performed on less than 1000 data points. This may cause instability."
/home/z1157095/anaconda3/en

iter 2 of 5


...

/home/z1157095/hypernet-cnn/deps/node/node/lib/odst.py:72: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  bin_codes = (indices.view(1, -1) // offsets.view(-1, 1) % 2).to(torch.float32)
/home/z1157095/hypernet-cnn/deps/node/node/lib/odst.py:113: UserWarning: Data-aware initialization is performed on less than 1000 data points. This may cause instability.To avoid potential problems, run this model on a data batch with at least 1000 data samples.You can do so manually before training. Use with torch.no_grad() for memory efficiency.
  warn("Data-aware initialization is performed on less than 1000 data points. This may cause instability."


iter 5 of 5


/home/z1157095/hypernet-cnn/deps/node/node/lib/odst.py:72: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  bin_codes = (indices.view(1, -1) // offsets.view(-1, 1) % 2).to(torch.float32)
/home/z1157095/hypernet-cnn/deps/node/node/lib/odst.py:113: UserWarning: Data-aware initialization is performed on less than 1000 data points. This may cause instability.To avoid potential problems, run this model on a data batch with at least 1000 data samples.You can do so manually before training. Use with torch.no_grad() for memory efficiency.
  warn("Data-aware initialization is performed on less than 1000 data points. This may cause instability."


====================== Summary ======================
Mode              : Best f : Steps : Time            
----------------  : ----   : ----  : ----            
Initial solution  : 50.62  : 1     : 05:46 (m:s)     
Random seeding    : 51.64  : 14    : 01:36:22 (h:m:s)
Local sampling    : 51.41  : 18    : 01:15:21 (h:m:s)
Duplicates        : -      : 170   : -               
----------------  : ----   : ----  : ----            
Total             : 51.64  : 203   : 02:57:29 (h:m:s)
Hill-Valley-with-noise_node_fn_{'layer_dim': 64, 'num_layers': 1, 'depth': 2}


{'layer_dim': 64, 'num_layers': 1, 'depth': 2}

#### Use best hyperparams

In [44]:
%%capture --no-stdout

size=max_size

node_results = test_model(node_fn(**node_best),
                    (X, y),
                    size,
                    label_encoder=None, iters=10)
res = node_results[node_results["Class"]=="roc_auc"].reset_index(drop=True)["Metric"]
print(f"{size}: {res.mean():.2f} ~ {res.std():.2f}, (max: {res.max():.2f})")

iter 1 of 10
iter 2 of 10
iter 3 of 10
iter 4 of 10
iter 5 of 10
iter 6 of 10
iter 7 of 10
iter 8 of 10
iter 9 of 10
iter 10 of 10
606: 51.82 ~ 2.84, (max: 59.34)


### Dropout Neural network

In [45]:
def network_fn1(epochs=100, drop1=0.3, drop2=0.5, batch_size=32, lr=3e-4):
    
    def _inner():
        network = torch.nn.Sequential(
                        torch.nn.Dropout(drop1),
                        torch.nn.Linear(n_features, 64),
                        torch.nn.ReLU(),
                        torch.nn.Dropout(drop2),
                        torch.nn.Linear(64, n_classes)
                    ).to(DEVICE).train()

        network = SimpleSklearnInterface(network, epochs=epochs, batch_size=batch_size, lr=lr)
        return network
    return _inner



def network_fn2(epochs=100, drop1=0.3, drop2=0.5, drop3=0.5, batch_size=32, lr=3e-4):
    
    def _inner():
        network = torch.nn.Sequential(
                        torch.nn.Dropout(drop1),
                        torch.nn.Linear(n_features, 64),
                        torch.nn.ReLU(),
                        torch.nn.Dropout(drop2),
                        torch.nn.Linear(64, 64),
                        torch.nn.ReLU(),
                        torch.nn.Dropout(drop3),
                        torch.nn.Linear(64, n_classes)
                    ).to(DEVICE).train()

        network = SimpleSklearnInterface(network, epochs=epochs, batch_size=batch_size, lr=lr)
        return network
    return _inner



def network_fn3(epochs=100, drop1=0.3, drop2=0.5, drop3=0.5, drop4=0.5, batch_size=32, lr=3e-4):
    
    def _inner():
        network = torch.nn.Sequential(
                        torch.nn.Dropout(drop1),
                        torch.nn.Linear(n_features, 64),
                        torch.nn.ReLU(),
                        torch.nn.Dropout(drop2),
                        torch.nn.Linear(64, 128),
                        torch.nn.ReLU(),
                        torch.nn.Dropout(drop3),
                        torch.nn.Linear(128, 64),
                        torch.nn.ReLU(),
                        torch.nn.Dropout(drop4),
                        torch.nn.Linear(64, n_classes)
                    ).to(DEVICE).train()

        network = SimpleSklearnInterface(network, epochs=epochs, batch_size=batch_size, lr=lr)
        return network
    return _inner

#### Find Hyperparams

### Dropout 1 layer

In [46]:
param_grid = {
                "epochs": pyhopper.choice([100, 150]),
                "drop1": pyhopper.choice([0.1, 0.3, 0.5, 0.7], is_ordinal=True),
                "drop2": pyhopper.choice([0.1, 0.3, 0.5, 0.7], is_ordinal=True),
                "lr": pyhopper.choice([3e-5, 3e-4, 3e-3, 3e-2, 3e-1]),
                "batch_size": pyhopper.choice([32, 64]),
             }

nn_fn1_best_params, nn_fn1_history = pyhopper_best_params(network_fn1, param_grid, time="60m")
nn_fn1_best_params

2023-01-12 01:34:21.485 | INFO     | __main__:pyhopper_best_params:7 - pyhopper_best_params network_fn1


  0%|          | [00:00<?]

Search is scheduled for 01:00:00 (h:m:s)
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5


/home/z1157095/anaconda3/envs/hyper-mnist/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5


...

iter 4 of 5


/home/z1157095/anaconda3/envs/hyper-mnist/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


iter 5 of 5
==================== Summary ===================
Mode              : Best f : Steps : Time       
----------------  : ----   : ----  : ----       
Initial solution  : 49.87  : 1     : 01:51 (m:s)
Random seeding    : 53.06  : 14    : 29:12 (m:s)
Local sampling    : 52.93  : 12    : 27:30 (m:s)
Duplicates        : -      : 17    : -          
----------------  : ----   : ----  : ----       
Total             : 53.06  : 44    : 58:33 (m:s)
Hill-Valley-with-noise_network_fn1_{'epochs': 150, 'drop1': 0.1, 'drop2': 0.3, 'lr': 0.0003, 'batch_size': 32}


/home/z1157095/anaconda3/envs/hyper-mnist/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


{'epochs': 150, 'drop1': 0.1, 'drop2': 0.3, 'lr': 0.0003, 'batch_size': 32}

In [47]:
data_size = max_size

nn1_results = test_model(network_fn1(**nn_fn1_best_params),
                (X, y),
                data_size,
                None, 10)

iter 1 of 10
iter 2 of 10
iter 3 of 10
iter 4 of 10
iter 5 of 10
iter 6 of 10
iter 7 of 10
iter 8 of 10
iter 9 of 10
iter 10 of 10


In [48]:
res = nn1_results[nn1_results["Class"]=="Balanced Acc score"]["Metric"]
print(f"{data_size}: {res.mean():.2f} ~ {res.std():.2f} (max: {res.max():.2f})")

606: nan ~ nan (max: nan)


### Dropout 2 layers

In [49]:
param_grid = {
                "epochs": pyhopper.choice([100, 150]),
                "drop1": pyhopper.choice([0.1, 0.3, 0.5, 0.7], is_ordinal=True),
                "drop2": pyhopper.choice([0.1, 0.3, 0.5, 0.7], is_ordinal=True),
                "drop3": pyhopper.choice([0.1, 0.3, 0.5, 0.7], is_ordinal=True),
                "lr": pyhopper.choice([3e-5, 3e-4, 3e-3, 3e-2, 3e-1]),
                "batch_size": pyhopper.choice([32, 64]),
             }

nn_fn2_best_params, nn_fn2_history = pyhopper_best_params(network_fn2, param_grid, time="70m")
nn_fn2_best_params

2023-01-12 02:38:26.939 | INFO     | __main__:pyhopper_best_params:7 - pyhopper_best_params network_fn2


  0%|          | [00:00<?]

Search is scheduled for 01:10:00 (h:m:s)
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5


/home/z1157095/anaconda3/envs/hyper-mnist/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


iter 2 of 5


...

/home/z1157095/anaconda3/envs/hyper-mnist/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


iter 1 of 5


/home/z1157095/anaconda3/envs/hyper-mnist/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
====================== Summary ======================
Mode              : Best f : Steps : Time            
----------------  : ----   : ----  : ----            
Initial solution  : 49.04  : 1     : 01:57 (m:s)     
Random seeding    : 52.03  : 16    : 33:33 (m:s)     
Local sampling    : 57.71  : 14    : 32:50 (m:s)     
Duplicates        : -      : 5     : -               
----------------  : ----   : ----  : ----            
Total             : 57.71  : 36    : 01:08:21 (h:m:s)
Hill-Valley-with-noise_network_fn2_{'epochs': 150, 'drop1': 0.1, 'drop2': 0.1, 'drop3': 0.1, 'lr': 0.003, 'batch_size': 64}


{'epochs': 150,
 'drop1': 0.1,
 'drop2': 0.1,
 'drop3': 0.1,
 'lr': 0.003,
 'batch_size': 64}

In [50]:
data_size = max_size

nn2_results = test_model(network_fn2(**nn_fn2_best_params),
                (X, y),
                data_size,
                None, 10)

iter 1 of 10
iter 2 of 10
iter 3 of 10
iter 4 of 10
iter 5 of 10
iter 6 of 10
iter 7 of 10
iter 8 of 10
iter 9 of 10
iter 10 of 10


### Dropout 3 layers

In [51]:
param_grid = {
                "epochs": pyhopper.choice([100, 150]),
                "drop1": pyhopper.choice([0.1, 0.3, 0.5, 0.7], is_ordinal=True),
                "drop2": pyhopper.choice([0.1, 0.3, 0.5, 0.7], is_ordinal=True),
                "drop3": pyhopper.choice([0.1, 0.3, 0.5, 0.7], is_ordinal=True),
                "drop4": pyhopper.choice([0.1, 0.3, 0.5, 0.7], is_ordinal=True),
                "lr": pyhopper.choice([3e-5, 3e-4, 3e-3, 3e-2, 3e-1]),
                "batch_size": pyhopper.choice([32, 64]),
             }

nn_fn3_best_params, nn_fn3_history = pyhopper_best_params(network_fn3, param_grid, time="75m")
nn_fn3_best_params

2023-01-12 03:51:50.262 | INFO     | __main__:pyhopper_best_params:7 - pyhopper_best_params network_fn3


  0%|          | [00:00<?]

Search is scheduled for 01:15:00 (h:m:s)
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5


/home/z1157095/anaconda3/envs/hyper-mnist/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


iter 5 of 5
iter 1 of 5
iter 2 of 5


...

/home/z1157095/anaconda3/envs/hyper-mnist/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


iter 1 of 5


/home/z1157095/anaconda3/envs/hyper-mnist/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
====================== Summary ======================
Mode              : Best f : Steps : Time            
----------------  : ----   : ----  : ----            
Initial solution  : 50.05  : 1     : 02:03 (m:s)     
Random seeding    : 51.59  : 16    : 36:40 (m:s)     
Local sampling    : 52.33  : 14    : 35:25 (m:s)     
Duplicates        : -      : 6     : -               
----------------  : ----   : ----  : ----            
Total             : 52.33  : 37    : 01:14:09 (h:m:s)
Hill-Valley-with-noise_network_fn3_{'epochs': 150, 'drop1': 0.1, 'drop2': 0.7, 'drop3': 0.1, 'drop4': 0.1, 'lr': 0.0003, 'batch_size': 64}


{'epochs': 150,
 'drop1': 0.1,
 'drop2': 0.7,
 'drop3': 0.1,
 'drop4': 0.1,
 'lr': 0.0003,
 'batch_size': 64}

In [52]:
data_size = max_size

nn3_results = test_model(network_fn3(**nn_fn3_best_params),
                (X, y),
                data_size,
                None, 10)

iter 1 of 10
iter 2 of 10
iter 3 of 10
iter 4 of 10
iter 5 of 10
iter 6 of 10
iter 7 of 10
iter 8 of 10
iter 9 of 10
iter 10 of 10


### HypernetworkPCA

In [32]:
def network_pca_fn(epochs=100, masks_no=100, mask_size=100, target_size=100, n_comp=5, lr=3e-4, batch_size=64, verbose=False):
    def _inner():
        hypernet = HypernetworkPCA(
                        target_architecture=[(mask_size, target_size), (target_size, n_classes)], 
                        test_nodes=masks_no,
                        architecture=torch.nn.Sequential(torch.nn.Linear(n_comp, 64), 
                            torch.nn.ReLU(),
                            torch.nn.Linear(64, 128),
                            torch.nn.ReLU(),
                            torch.nn.Dropout(),
                            torch.nn.Linear(128, 128),
                            torch.nn.ReLU(),
                        ),
                        mode=TrainingModes.CARTHESIAN,
                        input_size=n_features
                    ).to(DEVICE)    
        hypernet = hypernet.train()

        network = HypernetworkSklearnInterface(hypernet, device=DEVICE, epochs=epochs, batch_size=batch_size, verbose=verbose, lr=lr)
        return network
    return _inner

In [33]:
def figure_to_html():
    tmpfile = BytesIO()
    plt.gcf().savefig(tmpfile, format='png')
    encoded = base64.b64encode(tmpfile.getvalue()).decode('utf-8')

    html = '<img src=\'data:image/png;base64,{}\'>'.format(encoded)
    return html

#### Find hyperparams

In [34]:
param_grid = {
    "epochs": pyhopper.choice([100]),
    "masks_no": pyhopper.int(50, 200, multiple_of=10),
    "mask_size": pyhopper.int(3, 90),
    "target_size": pyhopper.choice([5, 10, 20, 50]),
    "n_comp": pyhopper.int(3, 50),
    "lr": pyhopper.choice([3e-5, 3e-4, 3e-3, 3e-2, 3e-1]),
    "batch_size": pyhopper.choice([32, 64]),

}

hp_pca_best_params, hp_pca_history = pyhopper_best_params(network_pca_fn, param_grid, time="360m")
hp_pca_best_params

2022-12-04 23:33:34.723 | INFO     | __main__:pyhopper_best_params:7 - pyhopper_best_params network_pca_fn


  0%|          | [00:00<?]

Search is scheduled for 06:00:00 (h:m:s)
iter 1 of 5
torch.Size([1, 128])
100
iter 2 of 5
torch.Size([1, 128])
100
iter 3 of 5
torch.Size([1, 128])
100
iter 4 of 5
torch.Size([1, 128])
100
iter 5 of 5
torch.Size([1, 128])
100
iter 1 of 5
torch.Size([1, 128])
100
iter 2 of 5
torch.Size([1, 128])
100
iter 3 of 5
torch.Size([1, 128])
100
iter 4 of 5
torch.Size([1, 128])
100
iter 5 of 5
torch.Size([1, 128])
100
iter 1 of 5
torch.Size([1, 128])
100
iter 2 of 5
torch.Size([1, 128])
100


/home/z1157095/anaconda3/envs/hyper-mnist/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


iter 3 of 5
torch.Size([1, 128])
100
iter 4 of 5
torch.Size([1, 128])
100


...

/home/z1157095/anaconda3/envs/hyper-mnist/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


iter 4 of 5
torch.Size([1, 128])
100


/home/z1157095/anaconda3/envs/hyper-mnist/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


iter 5 of 5
torch.Size([1, 128])
100
iter 1 of 5
torch.Size([1, 128])
100
iter 2 of 5
torch.Size([1, 128])
100
iter 3 of 5
torch.Size([1, 128])
100
iter 4 of 5
torch.Size([1, 128])
100
iter 5 of 5
torch.Size([1, 128])
100
====================== Summary ======================
Mode              : Best f : Steps : Time            
----------------  : ----   : ----  : ----            
Initial solution  : 73.88  : 1     : 01:05:25 (h:m:s)
Random seeding    : 67.07  : 2     : 02:20:54 (h:m:s)
Local sampling    : 65.45  : 2     : 01:39:32 (h:m:s)
----------------  : ----   : ----  : ----            
Total             : 73.88  : 5     : 05:05:51 (h:m:s)
Hill-Valley-with-noise_network_pca_fn_{'epochs': 100, 'masks_no': 130, 'mask_size': 46, 'target_size': 5, 'n_comp': 26, 'lr': 3e-05, 'batch_size': 32}


{'epochs': 100,
 'masks_no': 130,
 'mask_size': 46,
 'target_size': 5,
 'n_comp': 26,
 'lr': 3e-05,
 'batch_size': 32}

In [35]:
hp_pca_best_params

{'epochs': 100,
 'masks_no': 130,
 'mask_size': 46,
 'target_size': 5,
 'n_comp': 26,
 'lr': 3e-05,
 'batch_size': 32}

'Libras'
{'epochs': 150,
 'masks_no': 70,
 'mask_size': 20,
 'target_size': 10,
 'n_comp': 10}

'Lymphography' {'epochs': 120, 'masks_no': 50, 'mask_size': 4, 'target_size': 20, 'n_comp': 6}

In [36]:
DATA

'Hill-Valley-with-noise'

Ionosphere
{'epochs': 100, 'masks_no': 60, 'mask_size': 5, 'target_size': 10}


In [37]:
epochs = 100#hp_pca_best_params['epochs']
masks_no = hp_pca_best_params['masks_no']
mask_size = hp_pca_best_params['mask_size']
target_size = hp_pca_best_params['target_size']
n_comp = hp_pca_best_params['n_comp']
lr = hp_pca_best_params['lr']
batch_size = hp_pca_best_params['batch_size']
data_size = max_size

nn_pca_results = test_model(network_pca_fn(target_size=target_size, mask_size=mask_size, masks_no=masks_no, n_comp=n_comp, epochs=epochs),
                (X, y),
                data_size,
                None, 10)

# exp.log_table("metrics.csv", nn_pca_results.groupby("Class").mean())
# exp.log_metric("f1_score", nn_pca_results.groupby("Class").mean().loc["F1 score"])

iter 1 of 10
torch.Size([1, 128])
100
iter 2 of 10
torch.Size([1, 128])
100
iter 3 of 10
torch.Size([1, 128])
100
iter 4 of 10
torch.Size([1, 128])
100
iter 5 of 10
torch.Size([1, 128])
100
iter 6 of 10
torch.Size([1, 128])
100
iter 7 of 10
torch.Size([1, 128])
100
iter 8 of 10
torch.Size([1, 128])
100
iter 9 of 10
torch.Size([1, 128])
100
iter 10 of 10
torch.Size([1, 128])
100


In [38]:
res = nn_pca_results[nn_pca_results["Class"]=="Total"].reset_index(drop=True)["Metric"]
print(f"{data_size}: {res.mean():.2f} ~ {res.std():.2f} (max: {res.max():.2f})")

606: 68.02 ~ 4.22 (max: 78.88)


### Hypernetwork

In [53]:
def network_hp_fn(epochs=150, masks_no=100, mask_size=100, target_size=100, lr=3e-4, batch_size=64, verbose=False):
    def _inner():
        hypernet = Hypernetwork(
                        target_architecture=[(mask_size, target_size), (target_size, n_classes)],
                        test_nodes=masks_no,
                        architecture=torch.nn.Sequential(torch.nn.Linear(n_features, 64), 
                            torch.nn.ReLU(),
                            torch.nn.Linear(64, 128),
                            torch.nn.ReLU(),
                            torch.nn.Dropout(),
                            torch.nn.Linear(128, 128),
                            torch.nn.ReLU(),
                        ),
                        mode=TrainingModes.CARTHESIAN,
                    ).to(DEVICE)    
        hypernet = hypernet.train()

        network = HypernetworkSklearnInterface(hypernet, device=DEVICE, epochs=epochs, batch_size=batch_size, verbose=verbose, lr=lr)
        return network
    return _inner


#### Find hyperparams

In [54]:
param_grid = {
    "epochs": pyhopper.choice([100]),
    "masks_no": pyhopper.choice([50, 70, 100, 150, 200, 300]),
    "mask_size": pyhopper.choice([2, 5, 10, 20, 50, 90]),
    "target_size": pyhopper.choice([5, 10, 20, 50]),
    "lr": pyhopper.choice([3e-5, 3e-4, 3e-3, 3e-2, 3e-1]),
    "batch_size": pyhopper.choice([32, 64]),
}

hp_best_params, hp_history = pyhopper_best_params(network_hp_fn, param_grid, time="360m")
hp_best_params

2023-01-12 05:11:12.905 | INFO     | __main__:pyhopper_best_params:7 - pyhopper_best_params network_hp_fn


  0%|          | [00:00<?]

Search is scheduled for 06:00:00 (h:m:s)
iter 1 of 5
torch.Size([1, 128])
iter 2 of 5
torch.Size([1, 128])
iter 3 of 5
torch.Size([1, 128])
iter 4 of 5
torch.Size([1, 128])
iter 5 of 5
torch.Size([1, 128])
iter 1 of 5
torch.Size([1, 128])
iter 2 of 5
torch.Size([1, 128])


/home/z1157095/anaconda3/envs/hyper-mnist/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


iter 3 of 5
torch.Size([1, 128])


...

iter 4 of 5
torch.Size([1, 128])


/home/z1157095/anaconda3/envs/hyper-mnist/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


iter 5 of 5
torch.Size([1, 128])
====================== Summary ======================
Mode              : Best f : Steps : Time            
----------------  : ----   : ----  : ----            
Initial solution  : 51.19  : 1     : 25:29 (m:s)     
Random seeding    : 68.45  : 5     : 02:41:34 (h:m:s)
Local sampling    : 75.69  : 4     : 02:30:50 (h:m:s)
Duplicates        : -      : 1     : -               
----------------  : ----   : ----  : ----            
Total             : 75.69  : 11    : 05:37:53 (h:m:s)
Hill-Valley-with-noise_network_hp_fn_{'epochs': 100, 'masks_no': 50, 'mask_size': 90, 'target_size': 10, 'lr': 0.003, 'batch_size': 64}


/home/z1157095/anaconda3/envs/hyper-mnist/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


{'epochs': 100,
 'masks_no': 50,
 'mask_size': 90,
 'target_size': 10,
 'lr': 0.003,
 'batch_size': 64}

In [55]:
hp_best_params

{'epochs': 100,
 'masks_no': 50,
 'mask_size': 90,
 'target_size': 10,
 'lr': 0.003,
 'batch_size': 64}

#### Train using the best hyperparams

In [56]:
epochs = 100#hp_best_params['epochs']
masks_no = hp_best_params['masks_no']
mask_size = hp_best_params['mask_size']
target_size = hp_best_params['target_size']
data_size = max_size
batch_size = hp_best_params['batch_size']
lr = hp_best_params['lr']


hyper_results = test_model(network_hp_fn(epochs, masks_no, mask_size, target_size, batch_size=batch_size,verbose=True),
                    (X, y),
                    data_size,
                    None, 10)

iter 1 of 10
torch.Size([1, 128])


100%|██████████| 100/100 [02:50<00:00,  1.70s/it]


iter 2 of 10
torch.Size([1, 128])


100%|██████████| 100/100 [02:46<00:00,  1.67s/it]


iter 3 of 10
torch.Size([1, 128])


...

100%|██████████| 100/100 [02:49<00:00,  1.69s/it]


iter 9 of 10
torch.Size([1, 128])


100%|██████████| 100/100 [02:49<00:00,  1.70s/it]


iter 10 of 10
torch.Size([1, 128])


100%|██████████| 100/100 [02:51<00:00,  1.71s/it]


In [57]:
res = hyper_results[hyper_results["Class"]=="balanced_accuracy"].reset_index(drop=True)["Metric"]
print(f"{data_size}: {res.mean():.2f} ~ {res.std():.2f}, (max: {res.max():.2f})")

606: 70.16 ~ 3.25, (max: 74.25)


### Random forest

In [58]:
from sklearn.ensemble import RandomForestClassifier

In [59]:
def get_rf(**params):
    random_seed = np.random.randint(1024)
    def _inner():
        return RandomForestClassifier(
            random_state=random_seed,
            **params
        )
    return _inner

#### Find hyperparams

In [60]:
param_grid = {
                'n_estimators': pyhopper.int(50, 3000, multiple_of=50),
                'max_features': pyhopper.choice([None, 'sqrt', 0.2, 0.3, 0.5, 0.7]),
                'criterion' : pyhopper.choice(['gini', 'entropy']),
                'max_depth': pyhopper.choice([None, 2, 4, 8, 16]),
             }

rf_best, rf_history = pyhopper_best_params(get_rf, param_grid, time="90m")
rf_best

2023-01-12 11:17:25.854 | INFO     | __main__:pyhopper_best_params:7 - pyhopper_best_params get_rf


  0%|          | [00:00<?]

Search is scheduled for 01:30:00 (h:m:s)
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5

{'n_estimators': 1100,
 'max_features': 0.3,
 'criterion': 'entropy',
 'max_depth': None}

#### Use best params

In [61]:
size = max_size

rf_dframe = test_model(get_rf(**rf_best), 
                        (X, y),
                        size,
                        None, iters=10)

iter 1 of 10
iter 2 of 10
iter 3 of 10
iter 4 of 10
iter 5 of 10
iter 6 of 10
iter 7 of 10
iter 8 of 10
iter 9 of 10
iter 10 of 10


# Collect analytics

In [51]:
hyper_results.groupby("Class")['Metric'].agg(['mean', 'std', 'max'])

,mean,std,max
Class,,,
0,63.076923,38.567286,100.000000
1,72.052117,30.350818,100.000000
F1 score,66.904488,12.024056,77.179487
Precision,77.929296,20.356044,100.000000
Recall,72.052117,30.350818,100.000000
Total,67.623762,7.950455,78.877888
balanced_accuracy,67.564520,8.170018,79.105161
roc_auc,89.998311,14.137924,95.848267


In [49]:
nn_pca_results.groupby("Class")['Metric'].agg(['mean', 'std', 'max'])

,mean,std,max
Class,,,
0,92.207358,19.048251,99.665552
1,44.462541,20.862875,99.022801
F1 score,56.539422,10.621406,76.381910
Precision,92.363977,11.058494,98.750000
Recall,44.462541,20.862875,99.022801
Total,68.019802,4.223477,78.877888
balanced_accuracy,68.334949,4.158507,79.140022
roc_auc,88.946979,8.597384,96.682754


In [62]:
d = {}

In [63]:
d['Random forest'] = rf_dframe.groupby("Class")['Metric'].agg(['mean', 'std', 'max'])
d['Hypernet'] = hyper_results.groupby("Class")['Metric'].agg(['mean', 'std', 'max'])
# d['HypernetPCA'] = nn_pca_results.groupby("Class")['Metric'].agg(['mean', 'std', 'max'])
d['Dropout_1'] = nn1_results.groupby("Class")['Metric'].agg(['mean', 'std', 'max'])
d['Dropout_2'] = nn2_results.groupby("Class")['Metric'].agg(['mean', 'std', 'max'])
d['Dropout_3'] = nn3_results.groupby("Class")['Metric'].agg(['mean', 'std', 'max'])
d['Node'] = node_results.groupby("Class")['Metric'].agg(['mean', 'std', 'max'])
d['XGBoost'] = xgb_dframe.groupby("Class")['Metric'].agg(['mean', 'std', 'max'])

In [64]:
all_models_df=pd.concat(d, axis=0)
all_models_df

mean        std         max
              Class                                              
Random forest 0                  56.856187   0.000000   56.856187
              1                  53.745928   0.000000   53.745928
              F1 score           54.908486   0.000000   54.908486
              Precision          56.122449   0.000000   56.122449
              Recall             53.745928   0.000000   53.745928
              Total              55.280528   0.000000   55.280528
              balanced_accuracy  55.301058   0.000000   55.301058
              roc_auc            58.499559   0.000000   58.499559
Hypernet      0                  85.919732  27.330075   99.665552
              1                  54.397394  23.471863   99.348534
              F1 score           62.948483   7.722468   75.668790
              Precision          90.060001  15.600137   99.259259
              Recall             54.397394  23.471863   99.348534
              Total              69.950495   3.060056   73.927393
              balanced_accuracy  70.158563   3.247530   74.249670
              roc_auc            94.458292   4.731031   98.209014
Dropout_1     0                  75.752508   1.522464   77.926421
              1                  29.739414   2.466649   32.899023
              F1 score           38.736626   2.316584   41.753653
              Precision          55.701104   1.206584   58.139535
              Recall             29.739414   2.466649   32.899023
              Total              52.442244   0.781001   53.960396
              balanced_accuracy  52.745961   0.764507   54.246511
              roc_auc            54.003029   1.261795   56.252111
Dropout_2     0                  71.036789   8.405235   81.939799
              1                  41.954397   6.900961   57.003257
              F1 score           49.040313   3.740741   56.726094
              Precision          60.365627   3.808037   67.664671
              Recall             41.954397   6.900961   57.003257
              Total              56.303630   1.459046   59.075908
              balanced_accuracy  56.495593   1.506945   59.373808
              roc_auc            61.967797   3.889449   66.798122
Dropout_3     0                  56.923077  21.429519   68.896321
              1                  46.742671  20.325924  100.000000
              F1 score           47.991132   8.088480   67.250821
              Precision          53.060987   1.127013   54.411765
              Recall             46.742671  20.325924  100.000000
              Total              51.765677   0.695988   52.640264
              balanced_accuracy  51.832874   0.844200   52.777990
              roc_auc            52.602105   0.539112   53.510072
Node          0                  80.568562   2.106109   84.615385
              1                  21.791531   1.726692   24.104235
              F1 score           30.935959   1.718539   33.183857
              Precision          53.584345   1.562083   57.009346
              Recall             21.791531   1.726692   24.104235
              Total              50.792079   0.543414   51.815182
              balanced_accuracy  51.180046   0.551638   52.242546
              roc_auc            51.818494   2.844790   59.342216
XGBoost       0                  60.869565   0.000000   60.869565
              1                  54.723127   0.000000   54.723127
              F1 score           56.756757   0.000000   56.756757
              Precision          58.947368   0.000000   58.947368
              Recall             54.723127   0.000000   54.723127
              Total              57.755776   0.000000   57.755776
              balanced_accuracy  57.796346   0.000000   57.796346
              roc_auc            61.908315   0.000000   61.908315

In [65]:
os.environ['COMET_KEY']

'UXrV5UxyhTK3cyQNG6BDuc4bE'

In [66]:
all_models_df.to_csv(f"{DATA}_metrics.csv")

In [67]:
exp = Experiment(os.environ.get("COMET_KEY"), 'hypernet-uci-tune')
# exp.log_parameters({"epochs": epochs, "mask_size": mask_size, "masks_no": masks_no, "data_size": data_size})
exp.add_tag(f"hypernet-tune2{DATA}")
exp.log_table(f"{DATA}_metrics.csv", all_models_df)

COMET WARNING: Comet has disabled auto-logging functionality as it has been imported after the following ML modules: torch. Metrics and hyperparameters can still be logged using comet_ml.log_metrics() and comet_ml.log_parameters()
COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Experiment is live on comet.ml https://www.comet.com/abulenok/hypernet-uci-tune/0f53ad17688a4627b350c6eb9b706960



{'web': 'https://www.comet.com/api/asset/download?assetId=9f1d78a25fc44752bb191ffd674fe9dd&experimentKey=0f53ad17688a4627b350c6eb9b706960',
 'api': 'https://www.comet.com/api/rest/v2/experiment/asset/get-asset?assetId=9f1d78a25fc44752bb191ffd674fe9dd&experimentKey=0f53ad17688a4627b350c6eb9b706960',
 'assetId': '9f1d78a25fc44752bb191ffd674fe9dd'}

### Replace some data in existing

In [68]:
# tmp_df = pd.concat(d, axis=0)
# tmp_df = tmp_df.reset_index()
# tmp_df = tmp_df.rename(columns={tmp_df.columns[0]: DATA})

# tmp_df

In [69]:
# all_models_df = pd.read_csv(f"{DATA}_metrics.csv")
# all_models_df = all_models_df.rename(columns={all_models_df.columns[0]: DATA})
# all_models_df = all_models_df.drop(all_models_df[all_models_df.iloc[:, 0] == 'Hypernet'].index)
# all_models_df = all_models_df.drop(all_models_df[all_models_df.iloc[:, 0] == 'HypernetPCA'].index)

# all_models_df

In [70]:
# all_models_df = pd.concat([all_models_df, tmp_df])
# all_models_df

In [71]:
f"{DATA}_metrics.csv"

'Hill-Valley-with-noise_metrics.csv'

In [72]:
all_models_df = pd.read_csv(f"{DATA}_metrics.csv")

In [73]:
all_models_df.reset_index()

,index,Unnamed: 0,Class,mean,std,max
0,0,Random forest,0,56.856187,0.000000,56.856187
1,1,Random forest,1,53.745928,0.000000,53.745928
2,2,Random forest,F1 score,54.908486,0.000000,54.908486
3,3,Random forest,Precision,56.122449,0.000000,56.122449
4,4,Random forest,Recall,53.745928,0.000000,53.745928
5,5,Random forest,Total,55.280528,0.000000,55.280528
6,6,Random forest,balanced_accuracy,55.301058,0.000000,55.301058
7,7,Random forest,roc_auc,58.499559,0.000000,58.499559
8,8,Hypernet,0,85.919732,27.330075,99.665552
9,9,Hypernet,1,54.397394,23.471863,99.348534


In [74]:
# all_models_df = all_models_df.drop(all_models_df.columns[0], axis=1)
# all_models_df

In [75]:
tmp = all_models_df

In [76]:
tmp = tmp.rename(columns={tmp.columns[0]: DATA})

In [77]:
tmp[tmp['Class'] == "balanced_accuracy"]

,Hill-Valley-with-noise,Class,mean,std,max
6,Random forest,balanced_accuracy,55.301058,0.000000,55.301058
14,Hypernet,balanced_accuracy,70.158563,3.247530,74.249670
22,Dropout_1,balanced_accuracy,52.745961,0.764507,54.246511
30,Dropout_2,balanced_accuracy,56.495593,1.506945,59.373808
38,Dropout_3,balanced_accuracy,51.832874,0.844200,52.777990
46,Node,balanced_accuracy,51.180046,0.551638,52.242546
54,XGBoost,balanced_accuracy,57.796346,0.000000,57.796346
